In [3]:
import json
import pandas as pd
import os

In [4]:
df = pd.read_csv("cluster.csv")
print(len(df))
df = df.sort_values(by = 'total_nodes',ascending = True)

113


### Players vs Clusters

In [47]:
total_matches = df.sum(axis = 0, skipna = True)['total_matches']
total_clusters = len(df)
print("Total Matches: ",total_matches)
print("Total clusters: ",total_clusters)

matches_till_now=0
clusters_till_now =0

flag70 = True
flag90 = True
flag50 = True
flag30 = True

for ind, row in df.iterrows():
    clusters_till_now+=1
    matches_till_now+=row['total_matches']
    matches_percent = 100*matches_till_now/total_matches
    if(matches_percent >= 90 and flag90):
        cluster_percent = clusters_till_now/total_clusters
        print(f"Around 90% of the matches are played by {100 * cluster_percent}% clusters")
        flag90 = False
        flag70 = False
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 70 and flag70):
        cluster_percent = clusters_till_now/total_clusters
        print(f"Around 70% of the matches are played by {100 * cluster_percent}% clusters")
        flag70 = False
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 50 and flag50):
        cluster_percent = clusters_till_now/total_clusters
        print(f"Around 50% of the matches are played by {100 * cluster_percent}% clusters")
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 30 and flag30):
        cluster_percent = clusters_till_now/total_clusters
        print(f"Around 30% of the matches are played by {100 * cluster_percent}% clusters")
        flag30 = False
        continue


Total Matches:  4159.0
Total clusters:  113
Around 50% of the matches are played by 2.6548672566371683% clusters
Around 70% of the matches are played by 29.20353982300885% clusters
Around 90% of the matches are played by 100.0% clusters


### Cluster vs Nodes

In [5]:
total_clusters = len(df)

# Define the conditions
conditions = {
    "== 2     ": df[df['total_nodes'] == 2],
    "== 3     ": df[df['total_nodes'] == 3],
    "== 4     ": df[df['total_nodes'] == 4],
    "== 5     ": df[df['total_nodes'] == 5],
    ">5 & <=10": df[(df['total_nodes'] > 5) & (df['total_nodes'] <= 10)],
    ">10      ": df[df['total_nodes'] > 10]
}
print("Total Clusters: ",total_clusters)
# Calculate the percentage for each condition
for condition, subset in conditions.items():
    percentage = len(subset) / total_clusters * 100
    print(f"Clusters with players {condition}: {percentage:.2f}% & {len(subset)}clusters" )
    

Total Clusters:  113
Clusters with players == 2     : 54.87% & 62clusters
Clusters with players == 3     : 23.01% & 26clusters
Clusters with players == 4     : 8.85% & 10clusters
Clusters with players == 5     : 3.54% & 4clusters
Clusters with players >5 & <=10: 8.85% & 10clusters
Clusters with players >10      : 0.88% & 1clusters


### Cluster vs Matches

In [6]:
# Calculate the average matches played per person for each cluster size
average_matches_per_person = {}
for size in range(2, 12):
    cluster_size_df = df[df['total_nodes'] == size]
    average_matches_per_person[size] = cluster_size_df['total_matches'].mean() / size


# For cluster size between 5 and 10
cluster_size_df = df[(df['total_nodes'] >= 5) & (df['total_nodes'] <= 10)]
average_matches_per_person['(5,10]'] = cluster_size_df['total_matches'].mean() / cluster_size_df['total_nodes'].mean()

# For cluster size greater than 10
cluster_size_df = df[df['total_nodes'] > 10]
average_matches_per_person['(10,max]'] = cluster_size_df['total_matches'].mean() / cluster_size_df['total_nodes'].mean()

# Print the results
for size_range, average_matches in average_matches_per_person.items():
    print(f"Average matches played per person in a cluster having size {size_range}: {2*average_matches:.2f}")


Average matches played per person in a cluster having size 2: 6.47
Average matches played per person in a cluster having size 3: 11.69
Average matches played per person in a cluster having size 4: 12.00
Average matches played per person in a cluster having size 5: 14.90
Average matches played per person in a cluster having size 6: 12.83
Average matches played per person in a cluster having size 7: 27.33
Average matches played per person in a cluster having size 8: nan
Average matches played per person in a cluster having size 9: 25.11
Average matches played per person in a cluster having size 10: 30.20
Average matches played per person in a cluster having size 11: nan
Average matches played per person in a cluster having size (5,10]: 22.88
Average matches played per person in a cluster having size (10,max]: 36.33


In [51]:
with open('player_matches.json') as f:
    match_counts = json.load(f)

arr=[]
for key,value in match_counts.items():
    arr.append((value,key))
arr.sort(reverse=True)
# print(arr)


total_matches = df.sum(axis = 0, skipna = True)['total_matches']
print("Total Matches: ",total_matches)
total_players = len(match_counts)
print("Total Playes: ",total_players)

players_till_now=0
matches_till_now =0
# print(type(match_counts))
flag100 = True
flag70 = True
flag90 = True
flag50 = True
flag30 = True

for value,_ in arr:
    players_till_now +=1
    #print("matches_till_now: ",players_till_now)
    matches_till_now+=value
    matches_percent = 100*matches_till_now/total_matches
    
    if(matches_percent >= 200 and flag100):
        player_percent = players_till_now/total_players
        print(f"Around 200% of the matches are played by {100 * player_percent}% players")
        flag100 = False
        flag90 = False
        flag70 = False
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 90 and flag90):
        player_percent = players_till_now/total_players
        print(f"Around 90% of the matches are played by {100 * player_percent}% players")
        flag90 = False
        flag70 = False
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 70 and flag70):
        player_percent = players_till_now/total_players
        print(f"Around 70% of the matches are played by {100 * player_percent}% players")
        flag70 = False
        flag50 = False
        flag30 = False
        continue
    if(matches_percent >= 50 and flag50):
        player_percent = players_till_now/total_players
        print(f"Around 50% of the matches are played by {100 * player_percent}% players")
        flag50 = False
        flag30 = False
        continue

    if(matches_percent >= 30 and flag30) :
        player_percent = players_till_now/total_players
        print(f"Around 30% of the matches are played by {100 * player_percent}% players")
        flag30 = False
        continue





Total Matches:  4159.0
Total Playes:  445
Around 30% of the matches are played by 1.5730337078651686% players
Around 50% of the matches are played by 2.9213483146067416% players
Around 70% of the matches are played by 4.943820224719101% players
Around 90% of the matches are played by 7.865168539325842% players
Around 200% of the matches are played by 100.0% players


### Players vs Clusters

In [58]:
df = pd.read_csv("cluster.csv")

for ind,row in df.iterrows():
    for node in row["nodes"]:
        strr = row["nodes"].strip('{').strip('}')
        strr = strr.strip("'")
        lst = strr.split(',')
        for i in range(0,len(lst)):
            lst[i]=lst[i].strip(" ").strip("'")
            lst[i] = int(lst[i])
    df.at[ind,"nodes"]=lst

with open('player_matches.json') as f:
    match_counts = json.load(f)

# Function to calculate percentage of players with match counts exceeding a threshold
def calculate_percentage(df_slice, threshold):
    percent_players_played_above_threshold_matches=0
    for ind, row in df_slice.iterrows():
        total_matches = row['total_matches']
        total_players = row["total_nodes"]
        players_above_threshold = 0
        
        for node in row["nodes"]:
            if int(match_counts[str(node)]) >= threshold * total_matches:
                players_above_threshold += 1
                #print(threshold * total_matches,match_counts[str(node)])
        percent_players_played_above_threshold_matches+=100 * players_above_threshold/ total_players
    return percent_players_played_above_threshold_matches/len(df_slice)

cluster_analysis_discrete = {}
cluster_frequency = {}
print(f"Percentage of players with more than (value1)% matches: (value2)%")
ranges = {(2,5),(5,8),(8,11),(11,max(df['total_nodes'])+1)}
for lims in ranges:
    for cluster_size in range(lims[0],lims[1]):
        if len(df[df["total_nodes"]==cluster_size])==0:
            continue
        lst = []
        thresholds = [0,0.02,0.06,0.3]
        mean_percentages = {}
        for threshold in thresholds:
            percentages = []
            cluster_frequency[cluster_size]=len(df[df["total_nodes"]==cluster_size])
            percentages.append(calculate_percentage(df[df["total_nodes"]==cluster_size], threshold))  # change for cluster_size
            mean_percentages[threshold] = sum(percentages) / len(percentages)
        print(f"Cluster Size: {cluster_size}")
        for threshold, percentage in mean_percentages.items():
            print(f"Percentage of players with more than {threshold*100}% matches: {percentage:.2f}%")
        for iter in mean_percentages.items():
            lst.append(iter)
        cluster_analysis_discrete[cluster_size]=lst
    # print(cluster_analysis_discrete)

    cumulative_sums = {}
    counts = {}
    cluster_analysis_discrete
    # Iterate over each cluster size and each threshold to calculate the cumulative sum and count
    cluster_analysis_discrete=dict(sorted(cluster_analysis_discrete.items()))
    for cluster_size, thresholds_data in cluster_analysis_discrete.items():
        if cluster_size ==0:
            continue
        for threshold, percentage in thresholds_data:
            if threshold in cumulative_sums:
                cumulative_sums[threshold] += percentage*cluster_frequency[cluster_size]
                counts[threshold] += cluster_frequency[cluster_size]
            else:
                cumulative_sums[threshold] = percentage*cluster_frequency[cluster_size]
                counts[threshold] = cluster_frequency[cluster_size]

    # Calculate the average percentage for each threshold
    average_array = [(threshold*100, cumulative_sums[threshold] / counts[threshold]) for threshold in cumulative_sums]
    
    
    # print(f"\nAverage array for range ({lims[0],lims[1]}):", average_array)


Percentage of players with more than (value1)% matches: (value2)%
Cluster Size: 2
Percentage of players with more than 0% matches: 100.00%
Percentage of players with more than 2.0% matches: 100.00%
Percentage of players with more than 6.0% matches: 100.00%
Percentage of players with more than 30.0% matches: 100.00%
Cluster Size: 3
Percentage of players with more than 0% matches: 100.00%
Percentage of players with more than 2.0% matches: 100.00%
Percentage of players with more than 6.0% matches: 100.00%
Percentage of players with more than 30.0% matches: 88.46%
Cluster Size: 4
Percentage of players with more than 0% matches: 100.00%
Percentage of players with more than 2.0% matches: 100.00%
Percentage of players with more than 6.0% matches: 97.50%
Percentage of players with more than 30.0% matches: 77.50%
Cluster Size: 110
Percentage of players with more than 0% matches: 100.00%
Percentage of players with more than 2.0% matches: 24.55%
Percentage of players with more than 6.0% matches: 